In [1]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 
from typing import Any, List, Tuple, Dict, Optional, Union
from itertools import islice

import qube
%qubed

%load_ext autoreload
%autoreload 2

from qube.core.series import TimeSeries, OHLCV, time_as_nsec, Quote, Trade
from qube.data.readers import CsvDataReader, QuotesDataProcessor, OhlcvDataProcessor
from qube.ta.indicators import (sma, ema, tema, dema, kama, highest, lowest)
from qube.utils.pandas import scols
import numpy as np

 >  [dev] installed cython rebuilding hook

                   ╻
   ┏┓      ╻       ┃  ┏┓       Quantitative Backtesting Environment 
   ┃┃  ┓┏  ┣┓  ┏┓  ┃  ┏┛       
   ┗┻  ┗┻  ┗┛  ┗   ┃  ┗━       (c) 2024,  ver. Dev
                   ╹       



In [5]:
from qube.core.series import TimeSeries; from qube.ta.indicators import sma; del sys.modules['qube.core.series']; del sys.modules['qube.ta.indicators']
from qube.core.series import TimeSeries, OHLCV, time_as_nsec, Quote, Trade
from qube.ta.indicators import (sma, ema, tema, dema, kama, highest, lowest)

# Common design

## Standard CSV readers

Takes too much time for loading big data

In [4]:
r0 = CsvDataReader('data/quotes.csv', QuotesDataProcessor())
# r0.read()

In [5]:
r0.read('2017-08-24 13:09:29')

[[2017-08-24T13:09:29.000000000]	9.33000 (100.0) | 9.35000 (200.0),
 [2017-08-24T13:09:30.000000000]	9.36000 (100.0) | 9.39000 (200.0),
 [2017-08-24T13:09:31.000000000]	9.36000 (100.0) | 9.39000 (200.0)]

In [ ]:
# r0.read()

In [7]:
# r1 = CsvDataReader('data/BTCUSDT_ohlcv_M1.csv', OhlcvDataProcessor())
r1 = CsvDataReader('data/BTCUSDT_ohlcv_M1_sec.csv', OhlcvDataProcessor())
# r1 = CsvDataReader('data/AAPL.csv', OhlcvDataProcessor(), timestamp_parsers=["%m/%d/%Y", "%d-%m-%Y"])
# r1 = CsvDataReader('../../BTCUSDT_ohlcv_M1.csv.gz', OhlcvDataProcessor())

In [8]:
%%time
# data = r1.read('2024-01-08 11:05:00', '2024-02-08 18:00:00')
# data = r1.read('2024-01-08', '2024-01-08 00:59')
data = r1.read()
# data

CPU times: total: 125 ms
Wall time: 214 ms


In [ ]:
# data

In [9]:
print(str(data))

Test1[1Min] | 65178 records
  2024-02-15T06:17:00.000000000 {o:51965.900000 | h:52004.700000 | l:51950.000000 | c:51950.400000 | v:223.358000}
  2024-02-15T06:16:00.000000000 {o:51996.000000 | h:52014.700000 | l:51965.900000 | c:51966.000000 | v:261.885000}
  2024-02-15T06:15:00.000000000 {o:51983.300000 | h:52015.000000 | l:51983.200000 | c:51995.900000 | v:191.007000}
   .......... 
  2024-01-01T00:02:00.000000000 {o:42350.400000 | h:42370.800000 | l:42349.600000 | c:42360.200000 | v:271.521000}
  2024-01-01T00:01:00.000000000 {o:42331.900000 | h:42353.100000 | l:42331.800000 | c:42350.400000 | v:202.444000}
  2024-01-01T00:00:00.000000000 {o:42314.000000 | h:42335.800000 | l:42289.600000 | c:42331.900000 | v:289.641000}



In [223]:
t = np.datetime64('2023-01-01 12:00:12', 'ns') #.astype('datetime64[ns]').item()

In [224]:
t.item()

1672574412000000000

In [45]:
# mx = scols(data.close.pd(), ema(data.close, 100).pd(), sma(data.close, 50).pd())

In [ ]:
plt.plot(mx.close)
plt.plot(mx['ema(100,init_mean=True)'])
plt.plot(mx['sma(50)'])

In [49]:
# %%time
# dx0 = pd.read_csv('data/BTCUSDT_ohlcv_M1.csv', parse_dates=True, index_col=['time'], engine="pyarrow")

CPU times: total: 0 ns
Wall time: 17.6 ms


In [301]:
from pyarrow import csv
import pyarrow as pa
table = csv.read_csv(
    # 'data/BTCUSDT_ohlcv_M1_sec.csv', 
    # 'data/BTCUSDT_ohlcv_M1.csv', 
    'data/AAPL.csv', 
    # 'data/quotes.csv', 
    # parse_options=csv.ParseOptions(ignore_empty_lines=True),
    convert_options=csv.ConvertOptions(
        timestamp_parsers=["%d-%m-%Y"]
    )
)
table

pyarrow.Table
Date: string
Open: double
High: double
Low: double
Close: double
Volume: int64
----
Date: [["2-12-2016","1-12-2016","30-11-2016","29-11-2016","28-11-2016",...,"7-06-2016","6-06-2016","3-06-2016","2-06-2016","1-06-2016"]]
Open: [[109.17,110.36,111.6,110.78,111.43,...,99.25,97.99,97.79,97.6,99.02]]
High: [[110.09,110.94,112.2,112.03,112.46,...,99.87,101.89,98.27,97.84,99.54]]
Low: [[108.85,109.03,110.27,110.07,111.39,...,98.96,97.55,97.45,96.63,98.33]]
Close: [[109.9,109.49,110.52,111.46,111.57,...,99.03,98.63,97.92,97.72,98.46]]
Volume: [[26527997,37086862,36162258,28528750,27193983,...,22409450,23292504,28504888,40191600,29173285]]

In [297]:
import io
s = (
    "animals,n_legs,entry,fast\n"
    # "Flamingo,2,01/02/2022,Yes\n"
    "Flamingo,2,01-01-2022,Yes\n"
    "Flamingo,2,02/01/2022,Yes\n"
)
convert_options = csv.ConvertOptions(
                  timestamp_parsers=["%m/%d/%Y", "%d-%m-%Y"])
csv.read_csv(io.BytesIO(s.encode()), convert_options=convert_options)

pyarrow.Table
animals: string
n_legs: int64
entry: timestamp[s]
fast: string
----
animals: [["Flamingo","Flamingo"]]
n_legs: [[2,2]]
entry: [[2022-01-01 00:00:00,2022-02-01 00:00:00]]
fast: [["Yes","Yes"]]

In [294]:
pd.Timestamp('now').strftime("%d-%b-%Y")

'18-Feb-2024'

In [275]:
from datetime import datetime

In [277]:
datetime.strptime('12-May-2012', "%d-%b-%Y")

datetime.datetime(2012, 5, 12, 0, 0)

In [280]:
table

pyarrow.Table
Date: string
Open: double
High: double
Low: double
Close: double
Volume: int64
----
Date: [["2-Dec-2016","1-Dec-2016","30-Nov-2016","29-Nov-2016","28-Nov-2016",...,"11-Dec-2015","10-Dec-2015","9-Dec-2015","8-Dec-2015","7-Dec-2015"]]
Open: [[109.17,110.36,111.6,110.78,111.43,...,115.19,116.04,115.3,117.52,118.32]]
High: [[110.09,110.94,112.2,112.03,112.46,...,115.39,116.94,117.69,118.6,119.86]]
Low: [[108.85,109.03,110.27,110.07,111.39,...,112.85,115.51,115.08,116.86,117.81]]
Close: [[109.9,109.49,110.52,111.46,111.57,...,113.18,116.17,115.62,118.23,118.28]]
Volume: [[26527997,37086862,36162258,28528750,27193983,...,46886161,29212727,46361357,34309450,32084249]]

In [234]:
ts = table.field('time')
ts.type.unit

KeyError: 'Column time does not exist in schema'

In [55]:
ti = table['time']
table

pyarrow.Table
time: timestamp[ns]
open: double
high: double
low: double
close: double
volume: double
----
time: [[2024-01-01 00:00:00.000000000,2024-01-01 00:01:00.000000000,2024-01-01 00:02:00.000000000,2024-01-01 00:03:00.000000000,2024-01-01 00:04:00.000000000,...,2024-01-12 10:56:00.000000000,2024-01-12 10:57:00.000000000,2024-01-12 10:58:00.000000000,2024-01-12 10:59:00.000000000,2024-01-12 11:00:00.000000000],[2024-01-12 11:01:00.000000000,2024-01-12 11:02:00.000000000,2024-01-12 11:03:00.000000000,2024-01-12 11:04:00.000000000,2024-01-12 11:05:00.000000000,...,2024-01-23 22:39:00.000000000,2024-01-23 22:40:00.000000000,2024-01-23 22:41:00.000000000,2024-01-23 22:42:00.000000000,2024-01-23 22:43:00.000000000],[2024-01-23 22:44:00.000000000,2024-01-23 22:45:00.000000000,2024-01-23 22:46:00.000000000,2024-01-23 22:47:00.000000000,2024-01-23 22:48:00.000000000,...,2024-02-04 10:39:00.000000000,2024-02-04 10:40:00.000000000,2024-02-04 10:41:00.000000000,2024-02-04 10:42:00.000000000,

In [56]:
table[table.column_names[0]].num_chunks

4

In [70]:
ti > time_as_nsec('2024-02-08 18:05:00')

TypeError: '>' not supported between instances of 'pyarrow.lib.ChunkedArray' and 'int'

In [72]:
# s0 = ti.index(time_as_nsec('2024-01-24 13:05:00'))
s1 = ti.index(time_as_nsec('2024-02-08 18:05:00'))
s1
# print(s0, s1.as_py() - s0.as_py())

<pyarrow.Int64Scalar: -1>

In [101]:
for c in ti.iterchunks():
    a = c.to_numpy()
    ix = np.searchsorted(a, time_as_nsec('2024-02-08 10:01:01'))
    if ix < len(c):
        print(a[ix])
        break

2024-02-08T10:02:00.000000000


In [103]:
pa.ChunkedArray

pyarrow.lib.ChunkedArray

In [102]:
ti

[
  [
    2024-01-01 00:00:00.000000000,
    2024-01-01 00:01:00.000000000,
    2024-01-01 00:02:00.000000000,
    2024-01-01 00:03:00.000000000,
    2024-01-01 00:04:00.000000000,
    ...
    2024-01-12 10:56:00.000000000,
    2024-01-12 10:57:00.000000000,
    2024-01-12 10:58:00.000000000,
    2024-01-12 10:59:00.000000000,
    2024-01-12 11:00:00.000000000
  ],
  [
    2024-01-12 11:01:00.000000000,
    2024-01-12 11:02:00.000000000,
    2024-01-12 11:03:00.000000000,
    2024-01-12 11:04:00.000000000,
    2024-01-12 11:05:00.000000000,
    ...
    2024-01-23 22:39:00.000000000,
    2024-01-23 22:40:00.000000000,
    2024-01-23 22:41:00.000000000,
    2024-01-23 22:42:00.000000000,
    2024-01-23 22:43:00.000000000
  ],
  [
    2024-01-23 22:44:00.000000000,
    2024-01-23 22:45:00.000000000,
    2024-01-23 22:46:00.000000000,
    2024-01-23 22:47:00.000000000,
    2024-01-23 22:48:00.000000000,
    ...
    2024-02-04 10:39:00.000000000,
    2024-02-04 10:40:00.000000000,
    2024-

In [93]:
np.searchsorted(c.to_numpy(), time_as_nsec('2024-01-12 10:01'))

16441

In [90]:
c

[
  2024-01-01 00:00:00.000000000,
  2024-01-01 00:01:00.000000000,
  2024-01-01 00:02:00.000000000,
  2024-01-01 00:03:00.000000000,
  2024-01-01 00:04:00.000000000,
  2024-01-01 00:05:00.000000000,
  2024-01-01 00:06:00.000000000,
  2024-01-01 00:07:00.000000000,
  2024-01-01 00:08:00.000000000,
  2024-01-01 00:09:00.000000000,
  ...
  2024-01-12 10:51:00.000000000,
  2024-01-12 10:52:00.000000000,
  2024-01-12 10:53:00.000000000,
  2024-01-12 10:54:00.000000000,
  2024-01-12 10:55:00.000000000,
  2024-01-12 10:56:00.000000000,
  2024-01-12 10:57:00.000000000,
  2024-01-12 10:58:00.000000000,
  2024-01-12 10:59:00.000000000,
  2024-01-12 11:00:00.000000000
]

In [57]:
selected_table = table.slice(s0.as_py(), s1.as_py() - s0.as_py())

In [61]:
n_chunks = selected_table[table.column_names[0]].num_chunks
n_chunks

for n in range(n_chunks):
    data = [selected_table[k].chunk(n).to_numpy() for k in range(selected_table.num_columns)]
    print(data[0][0])
    # break

2024-01-08T11:05:00.000000000
2024-01-12T11:01:00.000000000
2024-01-23T22:44:00.000000000
2024-02-04T10:44:00.000000000


In [ ]:
data

### Fast OHLC series creation

In [ ]:
s = OHLCV('Test1', '1Min')
s.from_data()

In [ ]:
# tt = dx0.index.map(lambda x: x.asm8.item())
# s.open.times.set_values(list(tt))
# s.open.values.set_values(list(dx0.open.values))

# s.high.values.set_values(list(dx0.high.values))
# s.high.times.set_values(list(tt))

# s.low.values.set_values(list(dx0.low.values))
# s.low.times.set_values(list(tt))

# s.close.values.set_values(list(dx0.close.values))
# s.close.times.set_values(list(tt))

# s.volume.values.set_values(list(dx0.volume.values))
# s.volume.times.set_values(list(tt))

# s.times.set_values(list(tt))

## High level loader design

In [ ]:
class Loader:
    def symbols(self) -> list:
        pass

    def timerange(self) -> tuple:
        pass

    def timeframes(self) -> list:
        pass

    def ohlc(self, timeframe: str, start=None, end=None) -> list:
        pass

    def quotes(self):
        pass

    def trades(self):
        pass

# KDB

# Misc utils tests

In [ ]:
from itertools import islice

def chunked_iterable(iterable, chunk_size):
    it = iter(iterable)
    while True:
        chunk = list(islice(it, chunk_size))
        if not chunk:
            break
        yield chunk

In [ ]:
for p in chunked_iterable(np.arange(1,100), 10):
    print(p)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
[21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
[31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
[41, 42, 43, 44, 45, 46, 47, 48, 49, 50]
[51, 52, 53, 54, 55, 56, 57, 58, 59, 60]
[61, 62, 63, 64, 65, 66, 67, 68, 69, 70]
[71, 72, 73, 74, 75, 76, 77, 78, 79, 80]
[81, 82, 83, 84, 85, 86, 87, 88, 89, 90]
[91, 92, 93, 94, 95, 96, 97, 98, 99]


In [ ]:
def csv_fast_detect_number_of_lines(file):
    chunk = 1024 * 1024 * 512 # Process 500 MB at a time.
    f = np.memmap(file)
    num_newlines = sum(np.sum(f[i:i+chunk] == ord('\n'))
    for i in range(0, len(f), chunk))
    del f
    print(num_newlines)

csv_fast_detect_number_of_lines('data/BTCUSDT_ohlcv_M1.csv')

2333541
